<a href="https://colab.research.google.com/github/gabrielcarcedo/ResNet-architectures-comparassion/blob/main/ResNet_Comparassion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook para comparar las arquitecturas clásiscas de Redes Neuronales Recurrentes:

> - ## ResNet-18
> - ## ResNet-34
> - ## ResNet-50
> - ## ResNet-101
> - ## ResNet-152

## <strong>Objetivo:</strong> Clasificar imágenes del dataset de <a href="https://www.cs.toronto.edu/~kriz/cifar.html">CIFAR-10</a>

![CIFAR-10 Dataset](https://production-media.paperswithcode.com/datasets/4fdf2b82-2bc3-4f97-ba51-400322b228b1.png)

> ### Presenta: Gabriel Carcedo Rodríguez
> ### Posgrado en Ciencias en Ingeniería de la Computación
> ### Universidad Nacional Autónoma de México


In [1]:
#@markdown # Paquetes Necesarios
#@markdown # Utilización del Framework <strong>Pytorch</strong>

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


In [2]:
#@markdown # Configuración

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
EPOCHS = 5  # Reducido para demostración rápida


In [3]:
#@markdown # Transformaciones para CIFAR-10

# Transformaciones para CIFAR-10
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet requiere 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


In [4]:
#@markdown # Cargar dataset CIFAR-10

# Cargar dataset CIFAR-10
dataset_train = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
dataset_test = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)


100%|██████████| 170M/170M [00:03<00:00, 46.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
#@markdown # Función para entrenar y evaluar un modelo

# Función de entrenamiento y evaluación
def train_and_evaluate(model_name, model):
    print(f"Entrenando {model_name}...")

    model.to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_losses = []
    val_losses = []

    # Entrenamiento
    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_losses.append(running_loss / len(train_loader))
        print(f"{model_name} - Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss / len(train_loader):.4f}")

        # Evaluación
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_losses.append(val_loss / len(test_loader))

    # Visualización de pérdida
    plt.figure(figsize=(8,5))
    plt.plot(range(1, EPOCHS+1), train_losses, label='Train Loss')
    plt.plot(range(1, EPOCHS+1), val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'Loss Convergence for {model_name}')
    plt.legend()
    plt.show()

    # Precisión
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"{model_name} - Precisión en CIFAR-10: {accuracy:.2f}%\n")
    return accuracy


In [6]:
#@markdown # Modelos ResNet disponibles en torchvision
def run_model(model_name, model_func):
    model = model_func(pretrained=False)  # Sin pesos preentrenados para evaluación justa
    model.fc = nn.Linear(model.fc.in_features, 10)  # Ajuste para 10 clases
    accuracy = train_and_evaluate(model_name, model)
    print(f"{model_name}: {accuracy:.2f}%")


In [ ]:
#@markdown # Ejecución de modelo <strong>ResNet-18</strong>

run_model("ResNet-18", models.resnet18)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Entrenando ResNet-18...


In [ ]:
#@markdown # Ejecución de modelo <strong>ResNet-34</strong>

run_model("ResNet-34", models.resnet34)

In [ ]:
#@markdown # Ejecución de modelo <strong>ResNet-50</strong>

run_model("ResNet-50", models.resnet50)

In [ ]:
#@markdown # Ejecución de modelo <strong>ResNet-101</strong>

run_model("ResNet-101", models.resnet101)

In [ ]:
#@markdown # Ejecución de modelo <strong>ResNet-152</strong>

run_model("ResNet-152", models.resnet152)